In [ ]:
'''
    Make sure handCapture's subdirectories contains dataset images to be converted to numpy format first. Images converted are Grayscaled Images.
'''

import os
import sys

os.chdir(sys.path[0])
print(os.getcwd())

import cv2 as cv
import numpy as np
from tqdm import tqdm

IMG_FOLDER = "./handCapture/"

import COV_NET5 as  NeuralNet
import importlib

# Reload module if there's any changes to NeuralNet
importlib.reload(NeuralNet)

BUILD_DATA = True 

# Histogram Equalization
def his_equ(read_img):
    # convert from BGR color-space to YCrCb
    ycrcb_img = cv.cvtColor(read_img, cv.COLOR_BGR2YCrCb)

    # equalize the histogram on the Y plane
    ycrcb_img[:, :, 0] = cv.equalizeHist(ycrcb_img[:, :, 0])

    # convert back to RGB color-space from YCrCb
    equalized_img = cv.cvtColor(ycrcb_img, cv.COLOR_YCrCb2BGR)
    equalized_img = cv.cvtColor(equalized_img, cv.COLOR_BGR2GRAY)

    return equalized_img


class PalmFist():

    IMG_WIDTH = NeuralNet.IMG_WIDTH
    IMG_HEIGHT = NeuralNet.IMG_HEIGHT
    training_data = []

    # ========================= CLASSES =========================

    AMT_OF_CLASS = NeuralNet.NUM_CLASSES
    
    PALM = IMG_FOLDER + "openPalm"
    FIST = IMG_FOLDER + "fist"
    PEACE = IMG_FOLDER + "peace"
    RIGHT= IMG_FOLDER + "right"
    LEFT= IMG_FOLDER + "left"
    THUMB= IMG_FOLDER + "thumbsUp"

    LABELS = {  
        PALM:0, FIST:1, PEACE:2, RIGHT:3, LEFT:4, THUMB:5
    }
    
    palmCount = 0
    fistCount = 0 
    leftCount = 0 
    rightCount = 0 
    thumbCount = 0 
    peaceCount = 0 

    # ========================= CLASSES =========================

    def make_training_data(self):
        for label in self.LABELS:

            for fName in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, fName)

                    img = cv.imread(path)
                    img = his_equ(cv.imread(path))
                    img = cv.resize(img, (self.IMG_WIDTH, self.IMG_HEIGHT))

                    self.training_data.append([np.array(img), np.eye(self.AMT_OF_CLASS)[self.LABELS[label]]])

                    if label == self.PALM:
                        self.palmCount += 1
                    elif label == self.FIST:
                        self.fistCount += 1
                    elif label == self.LEFT:    
                        self.leftCount += 1
                    elif label == self.RIGHT:
                        self.rightCount += 1
                    elif label == self.THUMB:
                        self.thumbCount += 1
                    elif label == self.PEACE:
                        self.peaceCount += 1
                
                except Exception as e:
                    print("EXCEPTION IMAGE:\n",e)
                    continue
        
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        
        print("Palm:", self.palmCount)
        print("Fist:", self.fistCount)
        print("Left:", self.leftCount)
        print("Right:", self.rightCount)
        print("Thumb:", self.thumbCount)
        print("Peace:", self.peaceCount)

if BUILD_DATA:
    trainer = PalmFist()
    print("IMG SIZE:", trainer.IMG_WIDTH, trainer.IMG_HEIGHT)

    trainer.make_training_data()

After creation of training data, load it in for testing to ensure no errors

In [2]:
data_created = np.load("training_data.npy", allow_pickle=True)
print(len(data_created))

In [ ]:
import matplotlib.pyplot as plt

# Choose test number
NUM = 600
print(data_created[NUM][0])

# Assuming training data is gray
plt.imshow(data_created[NUM][0], cmap="gray")
plt.show()
print(data_created[NUM][1])